In [ ]:
# fetch_ko_sec_financials.py
# -*- coding: utf-8 -*-

import os
import re
import json
import time
import argparse
from datetime import datetime
from typing import Dict, Any, Optional, List

import requests
import pandas as pd


SEC_COMPANYFACTS = "https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
SEC_SUBMISSIONS = "https://data.sec.gov/submissions/CIK{cik}.json"

# KO CIK (10 digits, leading zeros)
KO_CIK = "0000021344"


def sec_get(url: str, user_agent: str, sleep_s: float = 0.2) -> Dict[str, Any]:
    """
    SEC endpoints require a descriptive User-Agent.
    Keep a small delay to respect rate limits.
    """
    headers = {
        "User-Agent": user_agent,
        "Accept-Encoding": "gzip, deflate",
        "Host": "data.sec.gov",
    }
    r = requests.get(url, headers=headers, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"SEC request failed ({r.status_code}): {r.text[:200]}")
    time.sleep(sleep_s)
    return r.json()


def normalize_cik(cik: str) -> str:
    digits = re.sub(r"\D", "", cik)
    return digits.zfill(10)


def pick_annual_facts(
    companyfacts: Dict[str, Any],
    tag: str,
    unit_preference: Optional[List[str]] = None,
    form_filter: Optional[List[str]] = None,
    require_fp_fy: bool = True,
) -> pd.DataFrame:
    """
    Extract annual (FY) facts for a US-GAAP tag.
    We try to keep only 10-K / 20-F annual numbers when possible.
    """
    unit_preference = unit_preference or ["USD", "USD/shares", "shares", "pure"]
    form_filter = form_filter or ["10-K", "20-F"]

    gaap = companyfacts.get("facts", {}).get("us-gaap", {})
    if tag not in gaap:
        return pd.DataFrame(columns=["end", "fy", "fp", "form", "val", "tag", "unit"])

    data = gaap[tag].get("units", {})

    # choose a unit
    unit = None
    for u in unit_preference:
        if u in data:
            unit = u
            break
    if unit is None:
        # fallback: take first available unit
        unit = next(iter(data.keys()))

    rows = data[unit]
    df = pd.DataFrame(rows)

    # columns often include: end, fy, fp, form, filed, frame, val, etc.
    keep_cols = [c for c in ["end", "fy", "fp", "form", "filed", "val"] if c in df.columns]
    df = df[keep_cols].copy()
    df["tag"] = tag
    df["unit"] = unit

    # filter forms
    if "form" in df.columns:
        df = df[df["form"].isin(form_filter)]

    # keep annual FY points
    if require_fp_fy and "fp" in df.columns:
        df = df[df["fp"].astype(str).str.upper() == "FY"]

    # drop duplicates: multiple filings may exist; keep latest filed
    if "filed" in df.columns:
        df["filed_dt"] = pd.to_datetime(df["filed"], errors="coerce")
        df["end_dt"] = pd.to_datetime(df["end"], errors="coerce")
        df = df.sort_values(["end_dt", "filed_dt"]).drop_duplicates(subset=["end"], keep="last")
        df = df.drop(columns=["filed_dt", "end_dt"], errors="ignore")

    return df


def build_financial_statements(
    companyfacts: Dict[str, Any],
    years: int = 5,
) -> Dict[str, pd.DataFrame]:
    """
    Build simplified IS/BS/CF tables for the last `years` fiscal years.
    We map common KO-relevant tags; some items may be missing or reported under alternatives.
    """

    # ---- Tag mapping (common US-GAAP) ----
    # Income Statement
    is_tags = {
        "Revenue": ["Revenues", "SalesRevenueNet"],
        "COGS": ["CostOfGoodsAndServicesSold"],  # may be missing for some companies
        "GrossProfit": ["GrossProfit"],
        "OperatingIncome_EBIT": ["OperatingIncomeLoss"],
        "InterestExpense": ["InterestExpense"],
        "PretaxIncome": ["IncomeBeforeIncomeTaxes"],
        "NetIncome": ["NetIncomeLoss"],
        "DilutedEPS": ["EarningsPerShareDiluted"],
        "WeightedAvgSharesDiluted": ["WeightedAverageNumberOfDilutedSharesOutstanding"],
    }

    # Balance Sheet
    bs_tags = {
        "CashAndCashEquivalents": ["CashAndCashEquivalentsAtCarryingValue"],
        "TotalAssets": ["Assets"],
        "TotalLiabilities": ["Liabilities"],
        "TotalEquity": ["StockholdersEquity"],
        "LongTermDebt": ["LongTermDebtNoncurrent", "LongTermDebt"],
        "ShortTermDebt": ["DebtCurrent", "ShortTermBorrowings"],
    }

    # Cash Flow
    cf_tags = {
        "CFO": ["NetCashProvidedByUsedInOperatingActivities"],
        "Capex": ["PaymentsToAcquirePropertyPlantAndEquipment"],  # usually positive here; treat as outflow
        "DividendsPaid": ["PaymentsOfDividends"],
        "ShareRepurchase": ["PaymentsForRepurchaseOfCommonStock"],
    }

    def collect_best(tag_candidates: List[str], unit_pref: Optional[List[str]] = None) -> pd.DataFrame:
        for t in tag_candidates:
            df = pick_annual_facts(companyfacts, t, unit_preference=unit_pref)
            if not df.empty:
                return df
        return pd.DataFrame()

    # assemble annual series into wide tables
    def make_table(mapping: Dict[str, List[str]], table_name: str) -> pd.DataFrame:
        series = []
        for label, candidates in mapping.items():
            unit_pref = ["USD"] if label not in ("DilutedEPS", "WeightedAvgSharesDiluted") else None
            df = collect_best(candidates, unit_pref)
            if df.empty:
                continue
            # keep end and value
            tmp = df[["end", "fy", "val"]].copy()
            tmp = tmp.rename(columns={"val": label})
            series.append(tmp)

        if not series:
            return pd.DataFrame()

        out = series[0]
        for s in series[1:]:
            out = out.merge(s, on=["end", "fy"], how="outer")

        out["end"] = pd.to_datetime(out["end"], errors="coerce")
        out = out.sort_values("end").reset_index(drop=True)

        # keep last N years
        if "fy" in out.columns:
            out = out.sort_values("fy").dropna(subset=["fy"])
            out["fy"] = out["fy"].astype(int)
            out = out[out["fy"] >= (out["fy"].max() - (years - 1))]
            out = out.sort_values("fy").reset_index(drop=True)

        return out

    income = make_table(is_tags, "IncomeStatement")
    balance = make_table(bs_tags, "BalanceSheet")
    cashflow = make_table(cf_tags, "CashFlow")

    # derived fields
    if not balance.empty:
        if "LongTermDebt" in balance.columns and "ShortTermDebt" in balance.columns:
            balance["TotalDebt_Est"] = balance["LongTermDebt"].fillna(0) + balance["ShortTermDebt"].fillna(0)

    if not cashflow.empty and "Capex" in cashflow.columns:
        # Capex in this tag is usually cash outflow as a positive number; convert to negative for convention
        cashflow["Capex"] = -cashflow["Capex"].astype(float)

    # merged (by FY)
    merged = None
    for df in [income, balance, cashflow]:
        if df is None or df.empty:
            continue
        if merged is None:
            merged = df.copy()
        else:
            merged = merged.merge(df, on=["fy"], how="outer", suffixes=("", "_dup"))

    if merged is not None:
        # clean duplicate end columns if they appear
        if "end_dup" in merged.columns and "end" in merged.columns:
            merged["end"] = merged["end"].combine_first(merged["end_dup"])
            merged = merged.drop(columns=["end_dup"], errors="ignore")
        merged = merged.sort_values("fy").reset_index(drop=True)

    return {
        "income": income,
        "balance": balance,
        "cashflow": cashflow,
        "merged": merged if merged is not None else pd.DataFrame(),
    }


def main():
    parser = argparse.ArgumentParser(description="Fetch KO (Coca-Cola) 5Y financials from SEC XBRL companyfacts API.")
    parser.add_argument("--cik", default=KO_CIK, help="CIK (digits). Default is KO: 0000021344")
    parser.add_argument("--years", type=int, default=5, help="Number of fiscal years to keep (default 5).")
    parser.add_argument("--outdir", default="ko_sec_output", help="Output directory for CSV files.")
    parser.add_argument(
        "--user-agent",
        default=os.getenv("SEC_USER_AGENT", ""),
        help='SEC required User-Agent, e.g. "Your Name your.email@domain.com". You can also set env SEC_USER_AGENT.',
    )
    args = parser.parse_args()

    if not args.user_agent.strip():
        raise SystemExit(
            'ERROR: You must provide --user-agent or set env SEC_USER_AGENT.\n'
            'Example: export SEC_USER_AGENT="Zhaoyu Wang zhaoyu@example.com"'
        )

    cik = normalize_cik(args.cik)
    outdir = args.outdir
    os.makedirs(outdir, exist_ok=True)

    # Fetch companyfacts
    facts_url = SEC_COMPANYFACTS.format(cik=cik)
    companyfacts = sec_get(facts_url, user_agent=args.user_agent)

    # Build tables
    tables = build_financial_statements(companyfacts, years=args.years)

    # Save
    for name, df in tables.items():
        path = os.path.join(outdir, f"KO_{name}.csv")
        df.to_csv(path, index=False)
        print(f"Saved: {path}  rows={len(df)} cols={len(df.columns)}")

    # Quick preview
    merged = tables["merged"]
    if not merged.empty:
        print("\nMerged preview:")
        print(merged.tail(min(len(merged), args.years)).to_string(index=False))


if __name__ == "__main__":
    main()


In [2]:
# fetch_KO_income_statement.py
# --------------------------------
# Fetch Coca-Cola (KO) annual Income Statement from SEC XBRL

import requests
import pandas as pd
import time
import os

CIK = "0000021344"  # Coca-Cola
SEC_URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIK}.json"

HEADERS = {
    # ⚠️ 一定要改成你自己的真实信息
    "User-Agent": "Zhaoyu Wang zhaoyu@example.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "data.sec.gov"
}

def get_company_facts():
    r = requests.get(SEC_URL, headers=HEADERS)
    if r.status_code != 200:
        raise RuntimeError("SEC request failed")
    time.sleep(0.2)
    return r.json()

def extract_annual(tag):
    """Extract annual (10-K, FY) values for a GAAP tag"""
    facts = company_facts["facts"]["us-gaap"]
    if tag not in facts:
        return pd.DataFrame()

    units = facts[tag]["units"]
    unit = "USD" if "USD" in units else list(units.keys())[0]
    df = pd.DataFrame(units[unit])

    df = df[df["form"] == "10-K"]
    df = df[df["fp"] == "FY"]
    df = df[["fy", "end", "val"]]
    df = df.sort_values("fy").drop_duplicates("fy", keep="last")
    return df.rename(columns={"val": tag})

# ---------------- main ----------------

company_facts = get_company_facts()

# Income Statement GAAP tags
income_tags = {
    "Revenue": ["Revenues", "SalesRevenueNet"],
    "GrossProfit": ["GrossProfit"],
    "OperatingIncome": ["OperatingIncomeLoss"],
    "InterestExpense": ["InterestExpense"],
    "PretaxIncome": ["IncomeBeforeIncomeTaxes"],
    "NetIncome": ["NetIncomeLoss"],
    "DilutedEPS": ["EarningsPerShareDiluted"],
    "DilutedShares": ["WeightedAverageNumberOfDilutedSharesOutstanding"]
}

dfs = []

for label, tag_list in income_tags.items():
    for tag in tag_list:
        df = extract_annual(tag)
        if not df.empty:
            df = df.rename(columns={tag: label})
            dfs.append(df)
            break

# 合并成一张 Income Statement
income_statement = dfs[0]
for df in dfs[1:]:
    income_statement = income_statement.merge(df, on=["fy", "end"], how="outer")

# 只保留最近 5 年
income_statement = income_statement.sort_values("fy")
income_statement = income_statement.tail(5)

# 保存
os.makedirs("output", exist_ok=True)
income_statement.to_csv("output/KO_income_statement.csv", index=False)

print("Saved: output/KO_income_statement.csv")
print(income_statement)


Saved: output/KO_income_statement.csv
        fy         end       Revenue   GrossProfit  OperatingIncome  \
13  2021.0  2019-12-31  3.726600e+10           NaN              NaN   
14  2021.0  2021-12-31           NaN  2.329800e+10     1.030800e+10   
15  2022.0  2022-12-31  4.300400e+10  2.500400e+10     1.090900e+10   
16  2023.0  2023-12-31  4.575400e+10  2.723400e+10     1.131100e+10   
17  2024.0  2024-12-31  4.706100e+10  2.873700e+10     9.992000e+09   

    InterestExpense     NetIncome  DilutedShares  
13              NaN           NaN            NaN  
14     1.597000e+09  9.771000e+09   4.340000e+09  
15     8.820000e+08  9.542000e+09   4.350000e+09  
16     1.527000e+09  1.071400e+10   4.339000e+09  
17     1.656000e+09  1.063100e+10   4.320000e+09  


In [6]:
import os
import time
import requests
import pandas as pd

# Coca-Cola
CIK = "0000021344"
SEC_URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIK}.json"

HEADERS = {
    # ⚠️ 改成你自己的真实信息（必填）
    "User-Agent": "Your Name your.email@domain.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "data.sec.gov",
}

def sec_get_json(url: str) -> dict:
    r = requests.get(url, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"SEC request failed ({r.status_code}): {r.text[:200]}")
    time.sleep(0.2)
    return r.json()

def extract_annual_point_in_time(companyfacts: dict, tag: str) -> pd.DataFrame:
    """
    Extract annual 10-K FY values for a balance sheet (point-in-time) tag.
    Return only: fy + value
    """
    gaap = companyfacts.get("facts", {}).get("us-gaap", {})
    if tag not in gaap:
        return pd.DataFrame(columns=["fy", tag])

    units = gaap[tag].get("units", {})
    if not units:
        return pd.DataFrame(columns=["fy", tag])

    # Choose unit (usually USD)
    unit = "USD" if "USD" in units else list(units.keys())[0]
    df = pd.DataFrame(units[unit])

    # Filter annual 10-K FY
    if "form" in df.columns:
        df = df[df["form"] == "10-K"]
    if "fp" in df.columns:
        df = df[df["fp"].astype(str).str.upper() == "FY"]

    # Need at least fy + val, use filed only for "latest" selection
    keep = [c for c in ["fy", "val", "filed"] if c in df.columns]
    df = df[keep].copy()
    df = df.dropna(subset=["fy", "val"])

    # Keep latest filing per FY
    if "filed" in df.columns:
        df["filed_dt"] = pd.to_datetime(df["filed"], errors="coerce")
        df = df.sort_values(["fy", "filed_dt"]).drop_duplicates("fy", keep="last")
    else:
        df = df.sort_values("fy").drop_duplicates("fy", keep="last")

    df["fy"] = df["fy"].astype(int)
    df = df[["fy", "val"]].rename(columns={"val": tag})
    return df

# ---- 1) fetch companyfacts ----
companyfacts = sec_get_json(SEC_URL)

# ---- 2) choose balance sheet tags (with fallbacks) ----
# 你作业常用的 BS 核心项（足够做 leverage/ROA/ROE/净债务等）
BS_TAGS = {
    "CashAndCashEquivalents": ["CashAndCashEquivalentsAtCarryingValue"],
    "TotalCurrentAssets": ["AssetsCurrent"],
    "TotalAssets": ["Assets"],
    "TotalCurrentLiabilities": ["LiabilitiesCurrent"],
    "TotalLiabilities": ["Liabilities"],
    "TotalEquity": ["StockholdersEquity"],

    # debt
    "ShortTermDebt": ["DebtCurrent", "ShortTermBorrowings"],
    "LongTermDebt": ["LongTermDebtNoncurrent", "LongTermDebt"],
}

# ---- 3) extract + merge by FY only (most robust) ----
out = pd.DataFrame()

for label, candidates in BS_TAGS.items():
    got = False
    for tag in candidates:
        tmp = extract_annual_point_in_time(companyfacts, tag)
        if not tmp.empty and tmp[tag].notna().any():
            tmp = tmp.rename(columns={tag: label})
            out = tmp if out.empty else out.merge(tmp, on="fy", how="outer")
            got = True
            break

    # 如果某个科目完全抓不到，就给一列空值，保证表结构完整
    if not got:
        if out.empty:
            out = pd.DataFrame({"fy": []})
        out[label] = pd.NA

# ---- 4) derived fields ----
if "ShortTermDebt" in out.columns or "LongTermDebt" in out.columns:
    out["TotalDebt_Est"] = out.get("ShortTermDebt", 0).fillna(0) + out.get("LongTermDebt", 0).fillna(0)

if "CashAndCashEquivalents" in out.columns and "TotalDebt_Est" in out.columns:
    out["NetDebt_Est"] = out["TotalDebt_Est"].fillna(0) - out["CashAndCashEquivalents"].fillna(0)

# ---- 5) keep last 5 fiscal years ----
if not out.empty:
    out = out.sort_values("fy")
    out = out.tail(5).reset_index(drop=True)

# ---- 6) save + show ----
os.makedirs("output", exist_ok=True)
out.to_csv("output/KO_balance_sheet.csv", index=False)

print("Saved: output/KO_balance_sheet.csv")
print("Shape:", out.shape)

# Notebook 显示（如果你不是 Notebook，这行也不影响）
try:
    from IPython.display import display
    display(out)
except Exception:
    print(out.to_string(index=False))


Saved: output/KO_balance_sheet.csv
Shape: (5, 11)


,fy,CashAndCashEquivalents,TotalCurrentAssets,TotalAssets,TotalCurrentLiabilities,TotalLiabilities,TotalEquity,ShortTermDebt,LongTermDebt,TotalDebt_Est,NetDebt_Est
0,2020,6795000000,19240000000,87296000000,14601000000,<NA>,19299000000,2.183000e+09,4.012500e+10,4.230800e+10,3.551300e+10
1,2021,9684000000,22545000000,94354000000,19950000000,<NA>,22999000000,3.307000e+09,3.811600e+10,4.142300e+10,3.173900e+10
2,2022,9519000000,22591000000,92763000000,19724000000,<NA>,24105000000,2.373000e+09,3.637700e+10,3.875000e+10,2.923100e+10
3,2023,9366000000,26732000000,97703000000,23571000000,<NA>,25941000000,NaN,3.554700e+10,3.554700e+10,2.618100e+10
4,2024,10828000000,25997000000,100549000000,25249000000,<NA>,24856000000,NaN,NaN,0.000000e+00,-1.082800e+10


In [7]:
import os
import time
import requests
import pandas as pd

# Coca-Cola
CIK = "0000021344"
SEC_URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIK}.json"

HEADERS = {
    # ⚠️ 改成你自己的真实信息（必填）
    "User-Agent": "Your Name your.email@domain.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "data.sec.gov",
}

def sec_get_json(url: str) -> dict:
    r = requests.get(url, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"SEC request failed ({r.status_code}): {r.text[:200]}")
    time.sleep(0.2)
    return r.json()

def extract_annual_flow(companyfacts: dict, tag: str) -> pd.DataFrame:
    """
    Extract annual 10-K FY values for a cash flow (period) tag.
    Return only: fy + value
    """
    gaap = companyfacts.get("facts", {}).get("us-gaap", {})
    if tag not in gaap:
        return pd.DataFrame(columns=["fy", tag])

    units = gaap[tag].get("units", {})
    if not units:
        return pd.DataFrame(columns=["fy", tag])

    # Choose unit (usually USD)
    unit = "USD" if "USD" in units else list(units.keys())[0]
    df = pd.DataFrame(units[unit])

    # Filter annual 10-K FY
    if "form" in df.columns:
        df = df[df["form"] == "10-K"]
    if "fp" in df.columns:
        df = df[df["fp"].astype(str).str.upper() == "FY"]

    keep = [c for c in ["fy", "val", "filed"] if c in df.columns]
    df = df[keep].copy()
    df = df.dropna(subset=["fy", "val"])

    # Keep latest filing per FY
    if "filed" in df.columns:
        df["filed_dt"] = pd.to_datetime(df["filed"], errors="coerce")
        df = df.sort_values(["fy", "filed_dt"]).drop_duplicates("fy", keep="last")
    else:
        df = df.sort_values("fy").drop_duplicates("fy", keep="last")

    df["fy"] = df["fy"].astype(int)
    df = df[["fy", "val"]].rename(columns={"val": tag})
    return df

# ---- 1) fetch companyfacts ----
companyfacts = sec_get_json(SEC_URL)

# ---- 2) Cash Flow tags (with fallbacks) ----
# 这些是做 fundamental/DCF 最核心的一组
CF_TAGS = {
    "CFO": ["NetCashProvidedByUsedInOperatingActivities"],

    # Capex：SEC 的这个 tag 经常是“支付金额”（正数），我们会转成负数
    "Capex": ["PaymentsToAcquirePropertyPlantAndEquipment"],

    "CFI": ["NetCashProvidedByUsedInInvestingActivities"],
    "CFF": ["NetCashProvidedByUsedInFinancingActivities"],

    # 资本回报（可能有缺失/命名差异，所以用候选）
    "DividendsPaid": ["PaymentsOfDividends", "PaymentsOfDividendsCommonStock"],
    "ShareRepurchase": ["PaymentsForRepurchaseOfCommonStock"],

    # 有些公司会披露 D&A（如果抓得到，做 FCFF 很好用）
    "DepreciationAndAmortization": ["DepreciationDepletionAndAmortization", "DepreciationAndAmortization"],
}

# ---- 3) extract + merge by FY only ----
out = pd.DataFrame()

for label, candidates in CF_TAGS.items():
    got = False
    for tag in candidates:
        tmp = extract_annual_flow(companyfacts, tag)
        if not tmp.empty and tmp[tag].notna().any():
            tmp = tmp.rename(columns={tag: label})
            out = tmp if out.empty else out.merge(tmp, on="fy", how="outer")
            got = True
            break

    if not got:
        if out.empty:
            out = pd.DataFrame({"fy": []})
        out[label] = pd.NA

# ---- 4) conventions + derived ----
# Capex 通常在这个 tag 下是“付出去的钱”（正数），统一转成负数更适合建模
if "Capex" in out.columns:
    out["Capex"] = pd.to_numeric(out["Capex"], errors="coerce")
    out["Capex"] = -out["Capex"]

# 简单 FCF（常用口径）：FCF = CFO + Capex（Capex 已转负）
if "CFO" in out.columns and "Capex" in out.columns:
    out["FCF_Simple"] = pd.to_numeric(out["CFO"], errors="coerce") + pd.to_numeric(out["Capex"], errors="coerce")

# ---- 5) keep last 5 fiscal years ----
if not out.empty:
    out = out.sort_values("fy").tail(5).reset_index(drop=True)

# ---- 6) save + show ----
os.makedirs("output", exist_ok=True)
out.to_csv("output/KO_cash_flow.csv", index=False)

print("Saved: output/KO_cash_flow.csv")
print("Shape:", out.shape)

try:
    from IPython.display import display
    display(out)
except Exception:
    print(out.to_string(index=False))


Saved: output/KO_cash_flow.csv
Shape: (5, 9)


,fy,CFO,Capex,CFI,CFF,DividendsPaid,ShareRepurchase,DepreciationAndAmortization,FCF_Simple
0,2020,9844000000,-1177000000,-1477000000,-8070000000,7047000000,118000000,1536000000,8667000000
1,2021,12625000000,-1367000000,-2765000000,-6786000000,7252000000,111000000,1452000000,11258000000
2,2022,11018000000,-1484000000,-763000000,-10250000000,7616000000,1418000000,1260000000,9534000000
3,2023,11599000000,-1852000000,-3349000000,-8310000000,7952000000,2289000000,1128000000,9747000000
4,2024,6805000000,-2064000000,2524000000,-6910000000,8359000000,1795000000,1075000000,4741000000


In [8]:
import pandas as pd
import numpy as np

# ----------------------------
# 1. Load financial statements
# ----------------------------
income = pd.read_csv("output/KO_income_statement.csv")
balance = pd.read_csv("output/KO_balance_sheet.csv")
cashflow = pd.read_csv("output/KO_cash_flow.csv")

# Merge on fiscal year
df = income.merge(balance, on="fy", how="inner")
df = df.merge(cashflow, on="fy", how="inner")

df = df.sort_values("fy").reset_index(drop=True)

# ----------------------------
# 2. Helper: average balance items
# ----------------------------
def avg(series):
    return (series + series.shift(1)) / 2

# ----------------------------
# 3. Profitability ratios
# ----------------------------
df["GrossMargin"] = df.get("GrossProfit") / df["Revenue"]
df["OperatingMargin"] = df["OperatingIncome"] / df["Revenue"]
df["NetMargin"] = df["NetIncome"] / df["Revenue"]

df["AvgAssets"] = avg(df["TotalAssets"])
df["AvgEquity"] = avg(df["TotalEquity"])

df["ROA"] = df["NetIncome"] / df["AvgAssets"]
df["ROE"] = df["NetIncome"] / df["AvgEquity"]

# ----------------------------
# 4. Leverage & solvency
# ----------------------------
df["DebtToEquity"] = df["TotalDebt_Est"] / df["TotalEquity"]
df["DebtToAssets"] = df["TotalDebt_Est"] / df["TotalAssets"]

df["NetDebt"] = df["TotalDebt_Est"] - df["CashAndCashEquivalents"]

df["InterestCoverage"] = df["OperatingIncome"] / df["InterestExpense"]

# ----------------------------
# 5. Growth metrics
# ----------------------------
df["Revenue_YoY"] = df["Revenue"].pct_change()
df["NetIncome_YoY"] = df["NetIncome"].pct_change()
df["CFO_YoY"] = df["CFO"].pct_change()

# Revenue CAGR (over full window)
n_years = df["fy"].nunique() - 1
if n_years > 0:
    df["Revenue_CAGR"] = (df["Revenue"].iloc[-1] / df["Revenue"].iloc[0]) ** (1 / n_years) - 1
else:
    df["Revenue_CAGR"] = np.nan

# ----------------------------
# 6. Efficiency ratios
# ----------------------------
df["AssetTurnover"] = df["Revenue"] / df["AvgAssets"]
df["EquityMultiplier"] = df["AvgAssets"] / df["AvgEquity"]

# ----------------------------
# 7. Final ratios table
# ----------------------------
ratios_cols = [
    "fy",

    # Profitability
    "GrossMargin", "OperatingMargin", "NetMargin", "ROA", "ROE",

    # Leverage
    "DebtToEquity", "DebtToAssets", "NetDebt", "InterestCoverage",

    # Growth
    "Revenue_YoY", "NetIncome_YoY", "CFO_YoY", "Revenue_CAGR",

    # Efficiency
    "AssetTurnover", "EquityMultiplier",
]

ratios = df[ratios_cols]

# Save
ratios.to_csv("output/KO_financial_ratios.csv", index=False)

from IPython.display import display
display(ratios.round(4))


/var/folders/yn/22p3z99n05g0tbtst64n21d00000gn/T/ipykernel_4521/2469806075.py:49: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Revenue_YoY"] = df["Revenue"].pct_change()


,fy,GrossMargin,OperatingMargin,NetMargin,ROA,ROE,DebtToEquity,DebtToAssets,NetDebt,InterestCoverage,Revenue_YoY,NetIncome_YoY,CFO_YoY,Revenue_CAGR,AssetTurnover,EquityMultiplier
0,2021.0,NaN,NaN,NaN,NaN,NaN,1.8011,0.4390,3.173900e+10,NaN,NaN,NaN,NaN,0.0809,NaN,NaN
1,2021.0,NaN,NaN,NaN,0.1036,0.4248,1.8011,0.4390,3.173900e+10,6.4546,0.0000,NaN,0.0000,0.0809,NaN,4.1025
2,2022.0,0.5814,0.2537,0.2219,0.1020,0.4051,1.6076,0.4177,2.923100e+10,12.3685,0.1540,-0.0234,-0.1273,0.0809,0.4596,3.9724
3,2023.0,0.5952,0.2472,0.2342,0.1125,0.4282,1.3703,0.3638,2.618100e+10,7.4073,0.0639,0.1228,0.0527,0.0809,0.4804,3.8058
4,2024.0,0.6106,0.2123,0.2259,0.1072,0.4186,0.0000,0.0000,-1.082800e+10,6.0338,0.0286,-0.0077,-0.4133,0.0809,0.4748,3.9028


In [12]:
# =========================
# 7) FCFF (for DCF) -- robust version
# =========================

# 7.1 Determine tax rate (robust fallback)
# Priority:
# (1) PretaxIncome available -> effective tax from NetIncome/PretaxIncome
# (2) IncomeTaxExpenseBenefit available -> tax = tax_expense / pretax_income
# (3) fallback to statutory 21% (US federal), common for coursework

DEFAULT_TAX_RATE = 0.21

pretax_col = None
for c in ["PretaxIncome", "IncomeBeforeIncomeTaxes"]:
    if c in fin.columns:
        pretax_col = c
        break

# optional: try to fetch IncomeTaxExpenseBenefit into fin if you didn't already
# If not present, it will just be missing and we skip it.
tax_exp_col = None
for c in ["IncomeTaxExpenseBenefit", "IncomeTaxExpenseBenefitContinuingOperations"]:
    if c in fin.columns:
        tax_exp_col = c
        break

tax_rate = DEFAULT_TAX_RATE

if pretax_col is not None:
    pretax = pd.to_numeric(fin[pretax_col], errors="coerce")

    if tax_exp_col is not None:
        tax_exp = pd.to_numeric(fin[tax_exp_col], errors="coerce")
        eff = tax_exp / pretax
    else:
        net = pd.to_numeric(fin["NetIncome"], errors="coerce")
        eff = 1 - (net / pretax)

    # clean unreasonable values (negative pretax, outliers)
    eff = eff.replace([np.inf, -np.inf], np.nan)
    eff = eff.where((eff >= 0) & (eff <= 0.6))  # reasonable band for effective tax

    if np.nanmean(eff) == np.nanmean(eff):  # not all NaN
        tax_rate = float(np.nanmean(eff))

print(f"Tax rate used for FCFF: {tax_rate:.4f}  (pretax_col={pretax_col}, tax_exp_col={tax_exp_col})")

# 7.2 Ensure DA exists
if "DA" not in fin.columns:
    fin["DA"] = 0.0
fin["DA"] = pd.to_numeric(fin["DA"], errors="coerce").fillna(0)

# 7.3 Ensure Capex exists
fin["Capex"] = pd.to_numeric(fin.get("Capex", np.nan), errors="coerce")

# 7.4 ΔNWC (if not available, assume 0)
fin["DeltaNWC"] = 0.0

# 7.5 FCFF formula
# FCFF = EBIT*(1-T) + DA + Capex - ΔNWC   (Capex should be negative already)
fin["FCFF"] = (
    pd.to_numeric(fin["EBIT"], errors="coerce") * (1 - tax_rate)
    + fin["DA"]
    + fin["Capex"]
    - fin["DeltaNWC"]
)


Tax rate used for FCFF: 0.2100  (pretax_col=None, tax_exp_col=None)
